In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog, scrolledtext
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.inspection import partial_dependence
import threading
from datetime import datetime
import json
import os
import matplotlib.pyplot as plt
from matplotlib import cm
import warnings
import re
warnings.filterwarnings('ignore')

class ConcreteStrengthPredictor:
    def __init__(self):
        self.setup_style()
        self.setup_paths()
        self.load_data()
        self.train_model()
        self.create_gui()
        
    def setup_style(self):
        self.colors = {
            'primary': '#2E86AB',        # Steel Blue
            'secondary': '#A23B72',      # Maroon
            'accent': '#F18F01',         # Orange
            'background': '#F5F7FA',     # Light Gray Blue
            'card_bg': '#FFFFFF',        # White
            'success': '#28A745',        # Green
            'warning': '#FFC107',        # Yellow
            'danger': '#DC3545',         # Red
            'steel_blue': '#4682B4',     # Steel Blue
            'concrete_gray': '#757575',  # Medium Gray
            'rf_green': '#388E3C',       # Darker Random Forest Green
            'rf_brown': '#5D4037',       # Darker Brown for Random Forest
            'light_gray': '#E0E0E0',     # Light Gray
            'dark_gray': '#424242',      # Dark Gray
        }
        
        # Font settings
        self.fonts = {
            'title': ('Segoe UI', 16, 'bold'),
            'subtitle': ('Segoe UI', 12, 'bold'),
            'normal': ('Segoe UI', 10),
            'bold': ('Segoe UI', 10, 'bold'),
            'large': ('Segoe UI', 24, 'bold'),
            'xlarge': ('Segoe UI', 32, 'bold'),
        }
        
    def setup_paths(self):
        self.data_path = r"D:\2026 Work\My papers\Paper 1\Data.csv"
        self.save_dir = r"D:\2026 Work\My papers\LIME"
        os.makedirs(self.save_dir, exist_ok=True)
        self.history_file = os.path.join(self.save_dir, "prediction_history.json")
        
    def clean_column_names(self, columns):
        """Clean column names by removing special characters"""
        cleaned = []
        for col in columns:
            # Replace special characters with underscore
            cleaned_col = re.sub(r'[^a-zA-Z0-9_ ]', '_', str(col))
            # Replace multiple spaces/underscores with single underscore
            cleaned_col = re.sub(r'[ _]+', '_', cleaned_col)
            # Remove leading/trailing underscores
            cleaned_col = cleaned_col.strip('_')
            # If empty after cleaning, use generic name
            if cleaned_col == '':
                cleaned_col = f'feature_{len(cleaned)}'
            cleaned.append(cleaned_col)
        return cleaned
        
    def load_data(self):
        try:
            print(f"📂 Loading data from: {self.data_path}")
            self.df = pd.read_csv(self.data_path)
            print(f"✅ Dataset loaded successfully!")
            print(f"📐 Dataset shape: {self.df.shape}")
            
            # Clean column names - remove special characters
            self.df.columns = self.clean_column_names(self.df.columns)
            print(f"🔧 Cleaned columns: {self.df.columns.tolist()}")
            
            # Find target column (CS MPa)
            target_col = None
            possible_targets = ['CS_MPa', 'CS_MPa_', 'CS_MPa__', 'CS', 'Mpa', 'Compressive_strength', 
                              'MPa', 'compressive_strength', 'cs', 'mpa', 'CS_MPA']
            
            # Also check for columns containing "CS" and "MPa"
            for col in self.df.columns:
                if 'cs' in col.lower() and 'mpa' in col.lower():
                    target_col = col
                    break
            
            if target_col is None:
                for target in possible_targets:
                    if target in self.df.columns:
                        target_col = target
                        break

            if target_col is None:
                # Use last column as fallback
                target_col = self.df.columns[-1]
            
            self.target_name = target_col
            print(f"🎯 Target column: '{self.target_name}'")
            
            # Prepare features and target
            self.X = self.df.drop(columns=[target_col])
            self.y = self.df[target_col]
            self.feature_names = self.X.columns.tolist()
            
            print(f"\n📋 Features found ({len(self.feature_names)}):")
            for i, feature in enumerate(self.feature_names, 1):
                print(f"   {i:2}. {feature}")
            
            # Calculate statistics for each feature
            self.feature_stats = {}
            for feature in self.feature_names:
                self.feature_stats[feature] = {
                    'min': float(self.X[feature].min()),
                    'max': float(self.X[feature].max()),
                    'mean': float(self.X[feature].mean()),
                    'std': float(self.X[feature].std()),
                    'median': float(self.X[feature].median()),
                    'q1': float(self.X[feature].quantile(0.25)),
                    'q3': float(self.X[feature].quantile(0.75))
                }
            
            print(f"🔍 Feature statistics calculated for {len(self.feature_names)} features")
            
        except Exception as e:
            error_msg = f"Failed to load data: {str(e)}"
            print(f"❌ {error_msg}")
            messagebox.showerror("Data Loading Error", error_msg)
            raise
    
    def train_model(self):
        try:
            print("\n🤖 Training Random Forest model with hyperparameters:")
            print("  • Bootstrap: True")
            print("  • Max Depth: 10")
            print("  • Min Samples Leaf: 1")
            print("  • Min Samples Split: 2")
            print("  • N Estimators: 50")
            print("  • Random State: 42")
            
            # Random Forest with specified hyperparameters
            self.model = RandomForestRegressor(
                bootstrap=True,
                max_depth=10,
                min_samples_leaf=1,
                min_samples_split=2,
                n_estimators=50,
                random_state=42,
                n_jobs=-1,  # Use all available cores
                verbose=0
            )
            
            self.model.fit(self.X, self.y)
            print("✅ Random Forest model training completed!")
            
            # Feature importance
            self.feature_importance = pd.DataFrame({
                'feature': self.feature_names,
                'importance': self.model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            print(f"\n🏆 Top 5 Most Important Features:")
            for i, (_, row) in enumerate(self.feature_importance.head(5).iterrows(), 1):
                print(f"   {i}. {row['feature']:30} : {row['importance']:.4f}")
            
            # Performance metrics
            predictions = self.model.predict(self.X)
            self.r2_score = 1 - (np.sum((self.y - predictions) ** 2) / np.sum((self.y - np.mean(self.y)) ** 2))
            self.rmse = np.sqrt(np.mean((predictions - self.y) ** 2))
            self.mae = np.mean(np.abs(predictions - self.y))
            
            print(f"\n📈 Model Performance:")
            print(f"   • R² Score: {self.r2_score:.4f}")
            print(f"   • RMSE: {self.rmse:.2f} MPa")
            print(f"   • MAE: {self.mae:.2f} MPa")
            print(f"   • Model: Random Forest Regressor")
            
        except Exception as e:
            error_msg = f"Failed to train model: {str(e)}"
            print(f"❌ {error_msg}")
            messagebox.showerror("Model Training Error", error_msg)
            raise
    
    def create_gui(self):
        self.root = tk.Tk()
        self.root.title("Concrete Compressive Strength Predictor")
        self.root.geometry("1400x900")
        self.root.configure(bg=self.colors['background'])
        
        # Center window
        self.root.update_idletasks()
        width = self.root.winfo_width()
        height = self.root.winfo_height()
        x = (self.root.winfo_screenwidth() // 2) - (width // 2)
        y = (self.root.winfo_screenheight() // 2) - (height // 2)
        self.root.geometry(f'1400x900+{x}+{y}')
        
        # Create notebook for tabs
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Create tabs
        self.prediction_tab = self.create_prediction_tab()
        self.analysis_tab = self.create_analysis_tab()
        self.history_tab = self.create_history_tab()
        
        # Add tabs to notebook
        self.notebook.add(self.prediction_tab, text="📊 Prediction")
        self.notebook.add(self.analysis_tab, text="📈 Analysis")
        self.notebook.add(self.history_tab, text="📋 History")
        
        # Initialize prediction history
        self.prediction_history = []
        self.load_history()
        
        print("\n✅ GUI created successfully!")
    
    def create_prediction_tab(self):
        tab = ttk.Frame(self.notebook)
        
        # Header
        header_frame = ttk.Frame(tab)
        header_frame.pack(fill='x', padx=20, pady=(10, 0))
        
        header_label = tk.Label(header_frame,
                              text="Concrete Compressive Strength Prediction",
                              font=self.fonts['title'],
                              fg=self.colors['rf_brown'],
                              bg=self.colors['background'])
        header_label.pack(side='left')
        
        # Model info chip
        model_chip = tk.Label(header_frame,
                            text=f"Random Forest • R²={self.r2_score:.3f} • RMSE={self.rmse:.1f} MPa",
                            font=('Segoe UI', 9),
                            fg='white',
                            bg=self.colors['rf_green'],
                            padx=10,
                            pady=3,
                            bd=0,
                            relief='flat')
        model_chip.pack(side='right')
        
        # Main container
        main_container = ttk.Frame(tab)
        main_container.pack(fill='both', expand=True, padx=20, pady=20)
        
        # Left panel - Inputs
        left_panel = ttk.LabelFrame(main_container, text="INPUT PARAMETERS", padding=20)
        left_panel.pack(side='left', fill='both', expand=True, padx=(0, 10))
        
        # Right panel - Results
        right_panel = ttk.LabelFrame(main_container, text="PREDICTION RESULTS", padding=20)
        right_panel.pack(side='right', fill='both', expand=True, padx=(10, 0))
        
        # ========== LEFT PANEL: Input Parameters ==========
        
        # Create canvas with scrollbar for inputs
        input_canvas = tk.Canvas(left_panel, highlightthickness=0)
        input_scrollbar = ttk.Scrollbar(left_panel, orient="vertical", command=input_canvas.yview)
        input_frame = ttk.Frame(input_canvas)
        
        input_frame.bind("<Configure>", lambda e: input_canvas.configure(scrollregion=input_canvas.bbox("all")))
        input_canvas.create_window((0, 0), window=input_frame, anchor="nw")
        input_canvas.configure(yscrollcommand=input_scrollbar.set)
        
        input_canvas.pack(side="left", fill="both", expand=True)
        input_scrollbar.pack(side="right", fill="y")
        
        # Quick selection frame
        quick_frame = ttk.LabelFrame(input_frame, text="Quick Presets", padding=15)
        quick_frame.pack(fill='x', pady=(0, 20))
        
        # Preset buttons grid
        preset_buttons = [
            ("Low Strength", "Non-structural applications (< 20 MPa)", self.colors['danger']),
            ("Standard Concrete", "General purpose (25-35 MPa)", self.colors['primary']),
            ("Structural Concrete", "Structural applications (40-50 MPa)", self.colors['accent']),
            ("High Strength", "High-performance (> 50 MPa)", self.colors['success']),
        ]
        
        for i, (text, desc, color) in enumerate(preset_buttons):
            btn_frame = ttk.Frame(quick_frame)
            btn_frame.pack(fill='x', pady=(0, 10) if i < len(preset_buttons)-1 else 0)
            
            btn = tk.Button(btn_frame,
                          text=text,
                          font=self.fonts['bold'],
                          fg='white',
                          bg=color,
                          relief='flat',
                          padx=20,
                          pady=8,
                          cursor='hand2',
                          command=lambda t=text: self.load_preset_by_name(t))
            btn.pack(side='left')
            
            desc_label = tk.Label(btn_frame,
                                text=desc,
                                font=('Segoe UI', 8),
                                fg=self.colors['concrete_gray'],
                                bg=self.colors['card_bg'])
            desc_label.pack(side='left', padx=(10, 0))
        
        # Quick action buttons
        action_frame = ttk.Frame(quick_frame)
        action_frame.pack(fill='x', pady=(15, 0))
        
        ttk.Button(action_frame, text="🎲 Random Sample", 
                  command=self.fill_random_sample,
                  style='Secondary.TButton').pack(side='left', padx=2)
        ttk.Button(action_frame, text="📊 Load Best", 
                  command=self.fill_best_case,
                  style='Secondary.TButton').pack(side='left', padx=2)
        ttk.Button(action_frame, text="📉 Load Worst", 
                  command=self.fill_worst_case,
                  style='Secondary.TButton').pack(side='left', padx=2)
        ttk.Button(action_frame, text="🔄 Reset", 
                  command=self.clear_inputs,
                  style='Secondary.TButton').pack(side='left', padx=2)
        
        # Feature inputs section
        input_features_frame = ttk.LabelFrame(input_frame, text="FEATURE VALUES", padding=10)
        input_features_frame.pack(fill='both', expand=True)
        
        self.entries = {}
        
        # Show top 12 features
        top_features = self.feature_importance.head(12)['feature'].tolist()
        
        for i, feature in enumerate(top_features):
            feat_frame = ttk.Frame(input_features_frame)
            feat_frame.pack(fill='x', pady=8)
            
            stats = self.feature_stats[feature]
            
            # Label with importance indicator
            importance = self.feature_importance[self.feature_importance['feature'] == feature]['importance'].values[0]
            importance_color = self.get_importance_color(importance)
            
            label_frame = ttk.Frame(feat_frame)
            label_frame.pack(side='left', fill='x', expand=True)
            
            label = tk.Label(label_frame,
                           text=f"{feature}",
                           font=self.fonts['bold'],
                           fg=self.colors['dark_gray'],
                           anchor='w')
            label.pack(side='left')
            
            # Importance indicator
            if importance > 0.1:
                importance_label = tk.Label(label_frame,
                                          text=" ★",
                                          font=('Segoe UI', 10),
                                          fg=self.colors['warning'])
                importance_label.pack(side='left')
            
            # Value range info
            range_label = tk.Label(label_frame,
                                 text=f"  [{stats['min']:.1f} - {stats['max']:.1f}]",
                                 font=('Segoe UI', 8),
                                 fg=self.colors['concrete_gray'])
            range_label.pack(side='right')
            
            # Entry and slider frame
            control_frame = ttk.Frame(feat_frame)
            control_frame.pack(fill='x', pady=(5, 0))
            
            # Entry field
            entry = ttk.Entry(control_frame,
                            width=15,
                            font=self.fonts['normal'],
                            justify='center')
            entry.insert(0, f"{stats['mean']:.2f}")
            entry.pack(side='left', padx=(0, 10))
            
            # Slider for easy adjustment
            if stats['max'] - stats['min'] > 0:
                slider = ttk.Scale(control_frame,
                                 from_=stats['min'],
                                 to=stats['max'],
                                 value=stats['mean'],
                                 orient='horizontal',
                                 length=250,
                                 command=lambda v, f=feature: self.update_entry_from_slider(f, float(v)))
                slider.pack(side='left', fill='x', expand=True)
            
            self.entries[feature] = entry
        
        # Predict button at bottom
        predict_frame = ttk.Frame(input_features_frame)
        predict_frame.pack(fill='x', pady=(20, 0))
        
        self.predict_btn = tk.Button(predict_frame,
                                   text="PREDICT COMPRESSIVE STRENGTH",
                                   font=('Segoe UI', 12, 'bold'),
                                   fg='white',
                                   bg=self.colors['rf_green'],
                                   relief='flat',
                                   padx=30,
                                   pady=12,
                                   cursor='hand2',
                                   command=self.threaded_predict)
        self.predict_btn.pack(expand=True)
        
        # ========== RIGHT PANEL: Results ==========
        
        # Main result display
        result_display_frame = ttk.Frame(right_panel)
        result_display_frame.pack(fill='x', pady=(0, 20))
        
        self.result_var = tk.StringVar(value="—")
        result_label = tk.Label(result_display_frame,
                               textvariable=self.result_var,
                               font=('Segoe UI', 48, 'bold'),
                               fg=self.colors['rf_brown'])
        result_label.pack()
        
        unit_label = tk.Label(result_display_frame,
                             text="Megapascals (MPa)",
                             font=('Segoe UI', 12),
                             fg=self.colors['concrete_gray'])
        unit_label.pack()
        
        # Classification badge
        self.classification_var = tk.StringVar(value="")
        self.classification_label = tk.Label(result_display_frame,
                                           textvariable=self.classification_var,
                                           font=('Segoe UI', 14, 'bold'),
                                           fg='white',
                                           padx=20,
                                           pady=6,
                                           relief='flat')
        self.classification_label.pack(pady=15)
        
        # Model info card
        info_card = tk.Frame(right_panel, bg='white', relief='flat', bd=1)
        info_card.pack(fill='x', pady=(0, 20))
        
        # Performance metrics
        metrics_frame = ttk.Frame(info_card)
        metrics_frame.pack(fill='x', padx=15, pady=15)
        
        metrics = [
            ("R² Score", f"{self.r2_score:.4f}", self.colors['rf_green']),
            ("RMSE", f"{self.rmse:.2f} MPa", self.colors['primary']),
            ("MAE", f"{self.mae:.2f} MPa", self.colors['accent']),
        ]
        
        for i, (label, value, color) in enumerate(metrics):
            metric_frame = ttk.Frame(metrics_frame)
            metric_frame.pack(side='left', padx=10 if i < len(metrics)-1 else 0)
            
            label_lbl = tk.Label(metric_frame,
                               text=label,
                               font=('Segoe UI', 9),
                               fg=self.colors['concrete_gray'],
                               bg='white')
            label_lbl.pack()
            
            value_lbl = tk.Label(metric_frame,
                               text=value,
                               font=('Segoe UI', 11, 'bold'),
                               fg=color,
                               bg='white')
            value_lbl.pack()
        
        # Detailed info frame
        detail_frame = ttk.LabelFrame(right_panel, text="DETAILED ANALYSIS", padding=15)
        detail_frame.pack(fill='both', expand=True)
        
        # Create notebook for detailed results
        detail_notebook = ttk.Notebook(detail_frame)
        detail_notebook.pack(fill='both', expand=True)
        
        # Tab 1: Feature Contributions
        contributions_tab = ttk.Frame(detail_notebook)
        detail_notebook.add(contributions_tab, text="Feature Contributions")
        
        self.contributions_text = scrolledtext.ScrolledText(contributions_tab,
                                                          height=8,
                                                          font=('Consolas', 9),
                                                          wrap=tk.WORD,
                                                          bg='#FAFAFA',
                                                          relief='flat')
        self.contributions_text.pack(fill='both', expand=True, padx=2, pady=2)
        self.contributions_text.insert(1.0, "Feature contributions will appear here after prediction.\n")
        self.contributions_text.config(state='disabled')
        
        # Tab 2: Model Info
        info_tab = ttk.Frame(detail_notebook)
        detail_notebook.add(info_tab, text="Model Information")
        
        info_text = f"""MODEL: Random Forest Regressor
TARGET: {self.target_name}
TRAINING SAMPLES: {len(self.X)}
FEATURES: {len(self.feature_names)}

HYPERPARAMETERS:
• Bootstrap: True
• Max Depth: 10
• Min Samples Leaf: 1
• Min Samples Split: 2
• N Estimators: 50
• Random State: 42

TOP FEATURES BY IMPORTANCE:
"""
        for i, (_, row) in enumerate(self.feature_importance.head(8).iterrows(), 1):
            info_text += f"{i}. {row['feature'][:30]:30} : {row['importance']:.4f}\n"
        
        info_label = tk.Label(info_tab,
                            text=info_text,
                            font=('Consolas', 9),
                            justify='left',
                            bg='#FAFAFA')
        info_label.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Action buttons
        action_frame = ttk.Frame(right_panel)
        action_frame.pack(fill='x', pady=(15, 0))
        
        btn_style = {'style': 'Action.TButton'}
        ttk.Button(action_frame, text="💾 Save Result", 
                  command=self.save_result, **btn_style).pack(side='left', padx=5)
        ttk.Button(action_frame, text="📋 Copy", 
                  command=self.copy_to_clipboard, **btn_style).pack(side='left', padx=5)
        ttk.Button(action_frame, text="📤 Export", 
                  command=self.export_results, **btn_style).pack(side='left', padx=5)
        
        return tab
    
    def create_analysis_tab(self):
        tab = ttk.Frame(self.notebook)
        
        # Header
        header_frame = ttk.Frame(tab)
        header_frame.pack(fill='x', padx=20, pady=10)
        
        header_label = tk.Label(header_frame,
                              text="Model Analysis & Visualization",
                              font=self.fonts['title'],
                              fg=self.colors['rf_brown'],
                              bg=self.colors['background'])
        header_label.pack(side='left')
        
        # Analysis controls
        control_frame = ttk.LabelFrame(tab, text="Visualization Tools", padding=20)
        control_frame.pack(fill='x', padx=20, pady=10)
        
        # Button grid with improved styling
        button_grid = ttk.Frame(control_frame)
        button_grid.pack()
        
        analysis_buttons = [
            ("📊 Feature Importance", "View top influential features", self.plot_feature_importance),
            ("📈 Data Distribution", "Target variable histogram", self.plot_data_distribution),
            ("📉 Partial Dependence", "Feature effect on predictions", self.plot_partial_dependence),
            ("🔗 Correlation Matrix", "Feature relationships", self.plot_correlation_matrix),
            ("📋 Feature Statistics", "Statistical distributions", self.plot_feature_statistics),
            ("🎯 Model Performance", "Actual vs predicted", self.plot_actual_vs_predicted),
        ]
        
        for i, (text, tooltip, command) in enumerate(analysis_buttons):
            row, col = divmod(i, 3)
            btn_frame = ttk.Frame(button_grid)
            btn_frame.grid(row=row, column=col, padx=5, pady=5, sticky='nsew')
            
            btn = tk.Button(btn_frame,
                          text=text,
                          font=('Segoe UI', 10),
                          fg=self.colors['dark_gray'],
                          bg='white',
                          relief='flat',
                          padx=15,
                          pady=10,
                          cursor='hand2',
                          command=command)
            btn.pack(fill='both', expand=True)
            
            # Tooltip label
            tooltip_label = tk.Label(btn_frame,
                                   text=tooltip,
                                   font=('Segoe UI', 8),
                                   fg=self.colors['concrete_gray'],
                                   bg='white')
            tooltip_label.pack(fill='x')
        
        # Plot area
        self.analysis_frame = ttk.Frame(tab)
        self.analysis_frame.pack(fill='both', expand=True, padx=20, pady=(0, 20))
        
        # Initial plot
        self.plot_feature_importance()
        
        return tab
    
    def create_history_tab(self):
        tab = ttk.Frame(self.notebook)
        
        # Header
        header_frame = ttk.Frame(tab)
        header_frame.pack(fill='x', padx=20, pady=10)
        
        header_label = tk.Label(header_frame,
                              text="Prediction History",
                              font=self.fonts['title'],
                              fg=self.colors['rf_brown'],
                              bg=self.colors['background'])
        header_label.pack(side='left')
        
        # History controls
        control_frame = ttk.LabelFrame(tab, text="History Management", padding=20)
        control_frame.pack(fill='x', padx=20, pady=10)
        
        # Control buttons with better styling
        button_frame = ttk.Frame(control_frame)
        button_frame.pack()
        
        control_buttons = [
            ("🗑️ Clear All", self.clear_history),
            ("💾 Export CSV", self.export_history),
            ("📤 Load History", self.load_history_from_file),
            ("🔄 Refresh", self.update_history_display),
        ]
        
        for text, command in control_buttons:
            btn = ttk.Button(button_frame, text=text, command=command, style='Action.TButton')
            btn.pack(side='left', padx=5)
        
        # Statistics
        self.history_stats_var = tk.StringVar(value="No predictions recorded yet")
        stats_label = tk.Label(control_frame, 
                              textvariable=self.history_stats_var,
                              font=('Segoe UI', 10),
                              fg=self.colors['rf_green'])
        stats_label.pack(pady=(10, 0))
        
        # History table
        table_frame = ttk.Frame(tab)
        table_frame.pack(fill='both', expand=True, padx=20, pady=(0, 20))
        
        # Create treeview for better presentation
        columns = ('Timestamp', 'Strength', 'Classification')
        self.history_tree = ttk.Treeview(table_frame, columns=columns, show='headings', height=15)
        
        # Define headings
        self.history_tree.heading('Timestamp', text='Timestamp')
        self.history_tree.heading('Strength', text='Strength (MPa)')
        self.history_tree.heading('Classification', text='Classification')
        
        # Define columns
        self.history_tree.column('Timestamp', width=200)
        self.history_tree.column('Strength', width=120, anchor='center')
        self.history_tree.column('Classification', width=200, anchor='center')
        
        # Add scrollbar
        scrollbar = ttk.Scrollbar(table_frame, orient='vertical', command=self.history_tree.yview)
        self.history_tree.configure(yscrollcommand=scrollbar.set)
        
        self.history_tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        
        # Bind double-click to load history entry
        self.history_tree.bind('<Double-Button-1>', self.load_history_entry)
        
        return tab
    
    def get_importance_color(self, importance):
        if importance > 0.1:
            return self.colors['danger']
        elif importance > 0.05:
            return self.colors['warning']
        else:
            return self.colors['rf_green']
    
    def load_preset_by_name(self, preset_name):
        # Define presets based on common concrete mixtures
        presets = {
            "Standard Concrete": {
                "description": "General purpose concrete (25-35 MPa)",
                "multiplier": 1.0
            },
            "High Strength": {
                "description": "High-performance concrete (> 50 MPa)",
                "multiplier": 1.3
            },
            "Structural Concrete": {
                "description": "Structural applications (40-50 MPa)",
                "multiplier": 1.2
            },
            "Low Strength": {
                "description": "Non-structural applications (< 20 MPa)",
                "multiplier": 0.7
            },
        }
        
        if preset_name in presets:
            multiplier = presets[preset_name]["multiplier"]
            
            for feature in self.entries:
                stats = self.feature_stats[feature]
                # Adjust based on feature importance
                importance = self.feature_importance[
                    self.feature_importance['feature'] == feature
                ]['importance'].values[0] if feature in self.feature_importance['feature'].values else 0
                
                # Important features get bigger adjustment
                adj_multiplier = multiplier * (1 + importance * 0.5)
                new_value = stats['mean'] * adj_multiplier
                
                # Ensure within bounds
                new_value = max(stats['min'], min(stats['max'], new_value))
                
                self.entries[feature].delete(0, tk.END)
                self.entries[feature].insert(0, f"{new_value:.2f}")
            
            messagebox.showinfo("Preset Loaded", 
                               f"Loaded '{preset_name}' preset\n{presets[preset_name]['description']}")
    
    def fill_random_sample(self):
        """Fill with a random sample from dataset"""
        random_idx = np.random.randint(0, len(self.df))
        sample = self.df.iloc[random_idx]
        
        for feature in self.entries:
            if feature in sample:
                self.entries[feature].delete(0, tk.END)
                value = sample[feature]
                self.entries[feature].insert(0, f"{value:.2f}")
        
        actual_strength = sample[self.target_name]
        messagebox.showinfo("Random Sample", 
                           f"Loaded random sample #{random_idx}\nActual strength: {actual_strength:.1f} MPa")
    
    def fill_best_case(self):
        """Fill with values that give highest predicted strength"""
        # Use the sample with highest actual strength
        best_idx = self.y.idxmax()
        sample = self.df.iloc[best_idx]
        
        for feature in self.entries:
            if feature in sample:
                self.entries[feature].delete(0, tk.END)
                self.entries[feature].insert(0, f"{sample[feature]:.2f}")
        
        messagebox.showinfo("Best Case", 
                           f"Loaded best case scenario\nMaximum strength in dataset: {self.y.max():.1f} MPa")
    
    def fill_worst_case(self):
        """Fill with values that give lowest predicted strength"""
        # Use the sample with lowest actual strength
        worst_idx = self.y.idxmin()
        sample = self.df.iloc[worst_idx]
        
        for feature in self.entries:
            if feature in sample:
                self.entries[feature].delete(0, tk.END)
                self.entries[feature].insert(0, f"{sample[feature]:.2f}")
        
        messagebox.showinfo("Worst Case", 
                           f"Loaded worst case scenario\nMinimum strength in dataset: {self.y.min():.1f} MPa")
    
    def update_entry_from_slider(self, feature, value):
        """Update entry field when slider moves"""
        if feature in self.entries:
            self.entries[feature].delete(0, tk.END)
            self.entries[feature].insert(0, f"{value:.2f}")
    
    def threaded_predict(self):
        """Run prediction in separate thread"""
        self.predict_btn.config(state='disabled', text="PREDICTING...")
        thread = threading.Thread(target=self.predict_strength)
        thread.daemon = True
        thread.start()
    
    def predict_strength(self):
        try:
            # Get input values
            inputs = {}
            for feature, entry in self.entries.items():
                value = entry.get().strip()
                if value:
                    try:
                        inputs[feature] = float(value)
                    except:
                        inputs[feature] = self.feature_stats[feature]['mean']
                else:
                    inputs[feature] = self.feature_stats[feature]['mean']
            
            # Fill missing features with their means
            for feature in self.feature_names:
                if feature not in inputs:
                    inputs[feature] = self.feature_stats[feature]['mean']
            
            # Create input data
            input_data = pd.DataFrame([inputs], columns=self.feature_names)
            
            # Make prediction
            prediction = self.model.predict(input_data)[0]
            
            # Calculate feature contributions
            feature_contributions = self.calculate_feature_contributions(inputs, prediction)
            
            # Update GUI
            self.root.after(0, lambda: self.display_result(
                prediction, inputs, feature_contributions))
            
        except Exception as e:
            self.root.after(0, lambda: messagebox.showerror(
                "Prediction Error", f"Prediction failed: {str(e)}"))
        
        self.root.after(0, self.enable_predict_button)
    
    def calculate_feature_contributions(self, inputs, prediction):
        """Calculate approximate feature contributions"""
        contributions = {}
        base_prediction = prediction
        
        for feature in self.feature_importance.head(5)['feature']:
            if feature in inputs:
                # Create modified input with feature set to mean
                modified_input = inputs.copy()
                modified_input[feature] = self.feature_stats[feature]['mean']
                
                # Predict with modified input
                modified_data = pd.DataFrame([modified_input], columns=self.feature_names)
                modified_prediction = self.model.predict(modified_data)[0]
                
                # Contribution is difference from base prediction
                contribution = base_prediction - modified_prediction
                contributions[feature] = contribution
        
        return contributions
    
    def enable_predict_button(self):
        self.predict_btn.config(state='normal', text="PREDICT COMPRESSIVE STRENGTH")
    
    def display_result(self, prediction, inputs, feature_contributions):
        # Format result
        result_text = f"{prediction:.1f}"
        self.result_var.set(result_text)
        
        # Determine classification
        classification, color = self.classify_strength(prediction)
        self.classification_var.set(classification)
        self.classification_label.config(bg=color)
        
        # Update feature contributions
        self.update_contributions_text(feature_contributions, prediction)
        
        # Save to history
        self.save_to_history(prediction, classification, inputs)
        
        # Success animation
        self.show_success_animation()
        
        # Auto-select Analysis tab to show feature importance
        self.notebook.select(1)  # Switch to Analysis tab
    
    def classify_strength(self, strength):
        if strength >= 60:
            return "HIGH-STRENGTH CONCRETE", self.colors['success']
        elif strength >= 40:
            return "STRUCTURAL CONCRETE", self.colors['primary']
        elif strength >= 25:
            return "STANDARD CONCRETE", self.colors['accent']
        elif strength >= 15:
            return "LIGHTWEIGHT CONCRETE", self.colors['warning']
        else:
            return "LOW-STRENGTH CONCRETE", self.colors['danger']
    
    def update_contributions_text(self, contributions, prediction):
        self.contributions_text.config(state='normal')
        self.contributions_text.delete(1.0, tk.END)
        
        if contributions:
            text = f"PREDICTION: {prediction:.1f} MPa\n"
            text += "="*50 + "\n\n"
            text += "FEATURE CONTRIBUTIONS:\n"
            text += "-"*30 + "\n\n"
            
            sorted_contrib = sorted(contributions.items(), key=lambda x: abs(x[1]), reverse=True)
            
            for feature, contribution in sorted_contrib:
                direction = "↑ Increased" if contribution > 0 else "↓ Decreased"
                effect = f"{abs(contribution):.1f} MPa"
                text += f"• {feature[:25]:25} {direction:12} by {effect}\n"
            
            self.contributions_text.insert(1.0, text)
        else:
            self.contributions_text.insert(1.0, "No contribution data available.")
        
        self.contributions_text.config(state='disabled')
    
    def show_success_animation(self):
        original_bg = self.predict_btn.cget('bg')
        self.predict_btn.configure(bg=self.colors['success'])
        self.root.after(1000, lambda: self.predict_btn.configure(bg=original_bg))
    
    def save_to_history(self, prediction, classification, inputs):
        history_entry = {
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'strength': float(prediction),
            'classification': classification,
            'model': 'Random Forest',
            **{k: float(inputs[k]) for k in self.feature_importance.head(5)['feature'].tolist() if k in inputs}
        }
        
        self.prediction_history.append(history_entry)
        self.update_history_display()
        self.save_history()
    
    def update_history_display(self):
        # Clear treeview
        for item in self.history_tree.get_children():
            self.history_tree.delete(item)
        
        # Add items from history
        for entry in self.prediction_history[-50:]:  # Last 50 entries
            self.history_tree.insert('', 'end', values=(
                entry['timestamp'],
                f"{entry['strength']:.1f}",
                entry['classification']
            ))
        
        # Update statistics
        if self.prediction_history:
            strengths = [h['strength'] for h in self.prediction_history]
            stats_text = (f"Predictions: {len(self.prediction_history)} | "
                         f"Avg: {np.mean(strengths):.1f} MPa | "
                         f"Range: {min(strengths):.1f}-{max(strengths):.1f} MPa")
            self.history_stats_var.set(stats_text)
    
    def load_history_entry(self, event):
        selection = self.history_tree.selection()
        if selection:
            item = self.history_tree.item(selection[0])
            values = item['values']
            
            # Find the corresponding entry in history
            for entry in self.prediction_history:
                if entry['timestamp'] == values[0] and f"{entry['strength']:.1f}" == values[1]:
                    # Fill inputs with historical values
                    for feature in self.entries:
                        if feature in entry:
                            self.entries[feature].delete(0, tk.END)
                            self.entries[feature].insert(0, f"{entry[feature]:.2f}")
                    
                    # Update result display
                    self.result_var.set(f"{entry['strength']:.1f}")
                    classification, color = self.classify_strength(entry['strength'])
                    self.classification_var.set(classification)
                    self.classification_label.config(bg=color)
                    
                    # Switch to prediction tab
                    self.notebook.select(0)
                    
                    messagebox.showinfo("History Loaded", 
                                       f"Loaded prediction from {entry['timestamp']}")
                    break
    
    def clear_inputs(self):
        for feature, entry in self.entries.items():
            entry.delete(0, tk.END)
            entry.insert(0, f"{self.feature_stats[feature]['mean']:.2f}")
        
        self.result_var.set("—")
        self.classification_var.set("")
        self.contributions_text.config(state='normal')
        self.contributions_text.delete(1.0, tk.END)
        self.contributions_text.insert(1.0, "Feature contributions will appear here after prediction.")
        self.contributions_text.config(state='disabled')
    
    def save_result(self):
        """Save current prediction result"""
        result_text = f"""CONCRETE COMPRESSIVE STRENGTH PREDICTION
========================================================================
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Predicted Strength: {self.result_var.get()} MPa
Classification: {self.classification_var.get()}

MODEL INFORMATION:
Algorithm: Random Forest Regressor
R² Score: {self.r2_score:.4f}
RMSE: {self.rmse:.2f} MPa
Hyperparameters: bootstrap=True, max_depth=10, min_samples_leaf=1, 
                 min_samples_split=2, n_estimators=50

INPUT PARAMETERS:
"""
        
        for feature in self.entries:
            result_text += f"{feature}: {self.entries[feature].get()}\n"
        
        filename = filedialog.asksaveasfilename(
            defaultextension=".txt",
            filetypes=[("Text files", "*.txt"), ("All files", "*.*")],
            initialfile=f"concrete_prediction_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        )
        
        if filename:
            try:
                with open(filename, 'w') as f:
                    f.write(result_text)
                messagebox.showinfo("Save Successful", f"Result saved to {filename}")
            except Exception as e:
                messagebox.showerror("Save Failed", f"Failed to save: {str(e)}")
    
    def copy_to_clipboard(self):
        """Copy result to clipboard"""
        result_text = f"Predicted Concrete Strength: {self.result_var.get()} MPa ({self.classification_var.get()})"
        self.root.clipboard_clear()
        self.root.clipboard_append(result_text)
        messagebox.showinfo("Copied", "Result copied to clipboard!")
    
    def export_results(self):
        """Export detailed results"""
        self.export_history()
    
    def clear_history(self):
        if not self.prediction_history:
            messagebox.showinfo("No History", "Prediction history is already empty.")
            return
        
        if messagebox.askyesno("Clear History", "Clear all prediction history?"):
            self.prediction_history = []
            self.update_history_display()
            self.save_history()
            messagebox.showinfo("History Cleared", "Prediction history cleared.")
    
    def export_history(self):
        if not self.prediction_history:
            messagebox.showwarning("No Data", "No prediction history to export.")
            return
        
        filename = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx"), ("All files", "*.*")],
            initialfile=f"concrete_predictions_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        )
        
        if filename:
            try:
                df = pd.DataFrame(self.prediction_history)
                if filename.endswith('.xlsx'):
                    df.to_excel(filename, index=False)
                else:
                    df.to_csv(filename, index=False)
                messagebox.showinfo("Export Successful", f"History exported to {filename}")
            except Exception as e:
                messagebox.showerror("Export Failed", f"Failed to export: {str(e)}")
    
    def load_history_from_file(self):
        filename = filedialog.askopenfilename(
            filetypes=[("JSON files", "*.json"), ("CSV files", "*.csv"), ("All files", "*.*")]
        )
        
        if filename:
            try:
                if filename.endswith('.json'):
                    with open(filename, 'r') as f:
                        self.prediction_history = json.load(f)
                elif filename.endswith('.csv'):
                    self.prediction_history = pd.read_csv(filename).to_dict('records')
                
                self.update_history_display()
                messagebox.showinfo("Load Successful", 
                                   f"Loaded {len(self.prediction_history)} predictions")
            except Exception as e:
                messagebox.showerror("Load Failed", f"Failed to load: {str(e)}")
    
    def save_history(self):
        try:
            with open(self.history_file, 'w') as f:
                json.dump(self.prediction_history, f, indent=2)
        except Exception as e:
            print(f"Failed to save history: {e}")
    
    def load_history(self):
        try:
            if os.path.exists(self.history_file):
                with open(self.history_file, 'r') as f:
                    self.prediction_history = json.load(f)
                self.update_history_display()
        except:
            pass
    
    # ========== PLOTTING FUNCTIONS ==========
    
    def plot_feature_importance(self):
        self.clear_analysis_frame()
        
        fig = Figure(figsize=(12, 8), dpi=100, facecolor='white')
        ax = fig.add_subplot(111)
        
        # Plot top 15 features
        importance_df = self.feature_importance.head(15)
        y_pos = np.arange(len(importance_df))
        
        colors = cm.viridis(np.linspace(0.3, 0.9, len(importance_df)))
        bars = ax.barh(y_pos, importance_df['importance'], 
                      color=colors, edgecolor='black', height=0.7)
        
        ax.set_yticks(y_pos)
        ax.set_yticklabels(importance_df['feature'], fontsize=10)
        ax.set_xlabel('Importance Score', fontsize=12, fontweight='bold')
        ax.set_title(f'Feature Importance - Random Forest Model', 
                    fontsize=14, fontweight='bold', pad=20)
        ax.grid(True, alpha=0.3, axis='x', linestyle='--')
        
        # Add value labels
        for i, (bar, importance) in enumerate(zip(bars, importance_df['importance'])):
            ax.text(bar.get_width() + 0.001, bar.get_y() + bar.get_height()/2,
                   f'{importance:.3f}', va='center', fontsize=9, fontweight='bold')
        
        fig.tight_layout()
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        widget = canvas.get_tk_widget()
        widget.pack(fill='both', expand=True)
        
        self.current_plot = fig
    
    def plot_data_distribution(self):
        self.clear_analysis_frame()
        
        fig = Figure(figsize=(12, 8), dpi=100, facecolor='white')
        ax = fig.add_subplot(111)
        
        # Create histogram
        n, bins, patches = ax.hist(self.y, bins=30, alpha=0.7, 
                                  color=self.colors['rf_green'], 
                                  edgecolor='black', density=True)
        
        ax.set_xlabel(f'{self.target_name} (MPa)', fontsize=12, fontweight='bold')
        ax.set_ylabel('Density', fontsize=12, fontweight='bold')
        ax.set_title(f'Distribution of Concrete Strength', 
                    fontsize=14, fontweight='bold', pad=20)
        ax.grid(True, alpha=0.3, linestyle='--')
        
        # Add statistics box
        stats_text = (f'Mean: {self.y.mean():.1f} MPa\n'
                     f'Std Dev: {self.y.std():.1f} MPa\n'
                     f'Min: {self.y.min():.1f} MPa\n'
                     f'Max: {self.y.max():.1f} MPa\n'
                     f'Samples: {len(self.y)}')
        
        ax.text(0.98, 0.98, stats_text, transform=ax.transAxes,
               fontsize=10, verticalalignment='top', horizontalalignment='right',
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        fig.tight_layout()
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        widget = canvas.get_tk_widget()
        widget.pack(fill='both', expand=True)
        
        self.current_plot = fig
    
    def plot_partial_dependence(self):
        self.clear_analysis_frame()
        
        fig = Figure(figsize=(14, 10), dpi=100, facecolor='white')
        
        # Plot PDP for top 4 features
        top_features = self.feature_importance.head(4)['feature'].tolist()
        
        for i, feature in enumerate(top_features, 1):
            ax = fig.add_subplot(2, 2, i)
            
            try:
                pdp = partial_dependence(self.model, self.X, [feature], 
                                        grid_resolution=50)
                
                ax.plot(pdp['values'][0], pdp['average'][0], 
                       linewidth=2.5, color=self.colors['rf_brown'], alpha=0.8)
                
                ax.set_xlabel(feature, fontsize=10, fontweight='bold')
                ax.set_ylabel('Effect on Strength (MPa)', fontsize=10, fontweight='bold')
                ax.set_title(f'Partial Dependence: {feature[:20]}', fontsize=11, fontweight='bold')
                ax.grid(True, alpha=0.3, linestyle='--')
                
            except Exception as e:
                ax.text(0.5, 0.5, f"Error plotting", 
                       ha='center', va='center', fontsize=10, color='red')
        
        fig.suptitle(f'Partial Dependence Plots - Random Forest Model', 
                    fontsize=16, fontweight='bold', y=0.98)
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        widget = canvas.get_tk_widget()
        widget.pack(fill='both', expand=True)
        
        self.current_plot = fig
    
    def plot_correlation_matrix(self):
        self.clear_analysis_frame()
        
        fig = Figure(figsize=(14, 12), dpi=100, facecolor='white')
        ax = fig.add_subplot(111)
        
        # Calculate correlation for top features
        top_features = self.feature_importance.head(8)['feature'].tolist()
        if self.target_name in self.df.columns:
            corr_features = top_features + [self.target_name]
            corr_matrix = self.df[corr_features].corr()
            
            im = ax.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
            
            ax.set_xticks(np.arange(len(corr_features)))
            ax.set_yticks(np.arange(len(corr_features)))
            ax.set_xticklabels([f[:10] for f in corr_features], rotation=45, ha='right', fontsize=9)
            ax.set_yticklabels([f[:10] for f in corr_features], fontsize=9)
            
            ax.set_title(f'Feature Correlation Matrix', fontsize=14, fontweight='bold', pad=20)
            
            cbar = ax.figure.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            cbar.ax.set_ylabel('Correlation', rotation=-90, va="bottom", fontsize=10)
        
        fig.tight_layout()
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        widget = canvas.get_tk_widget()
        widget.pack(fill='both', expand=True)
        
        self.current_plot = fig
    
    def plot_feature_statistics(self):
        self.clear_analysis_frame()
        
        fig = Figure(figsize=(14, 10), dpi=100, facecolor='white')
        
        # Plot statistics for top 6 features
        top_features = self.feature_importance.head(6)['feature'].tolist()
        
        for i, feature in enumerate(top_features, 1):
            ax = fig.add_subplot(2, 3, i)
            
            data = self.X[feature].dropna()
            bp = ax.boxplot(data, vert=True, patch_artist=True, 
                           widths=0.6, showfliers=True)
            
            bp['boxes'][0].set_facecolor(self.colors['rf_green'])
            bp['boxes'][0].set_alpha(0.7)
            
            ax.set_ylabel('Value', fontsize=9, fontweight='bold')
            ax.set_title(f'{feature[:15]}', fontsize=10, fontweight='bold')
            ax.grid(True, alpha=0.3, axis='y', linestyle='--')
        
        fig.suptitle(f'Feature Distributions - Top 6 Features', 
                    fontsize=16, fontweight='bold', y=0.98)
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        widget = canvas.get_tk_widget()
        widget.pack(fill='both', expand=True)
        
        self.current_plot = fig
    
    def plot_actual_vs_predicted(self):
        self.clear_analysis_frame()
        
        fig = Figure(figsize=(12, 10), dpi=100, facecolor='white')
        ax = fig.add_subplot(111)
        
        predictions = self.model.predict(self.X)
        
        scatter = ax.scatter(self.y, predictions, 
                            alpha=0.6, 
                            s=50, 
                            c=self.colors['rf_green'],
                            edgecolors='white', 
                            linewidth=0.5)
        
        min_val = min(self.y.min(), predictions.min())
        max_val = max(self.y.max(), predictions.max())
        ax.plot([min_val, max_val], [min_val, max_val], 
               'r--', linewidth=2, alpha=0.7, label='Perfect Prediction')
        
        ax.set_xlabel(f'Actual {self.target_name} (MPa)', fontsize=12, fontweight='bold')
        ax.set_ylabel(f'Predicted {self.target_name} (MPa)', fontsize=12, fontweight='bold')
        ax.set_title(f'Model Performance - Actual vs Predicted', 
                    fontsize=14, fontweight='bold', pad=20)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(fontsize=10)
        
        # Add R² text
        r2 = self.r2_score
        ax.text(0.05, 0.95, f'R² = {r2:.4f}', transform=ax.transAxes,
               fontsize=12, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        fig.tight_layout()
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        widget = canvas.get_tk_widget()
        widget.pack(fill='both', expand=True)
        
        self.current_plot = fig
    
    def clear_analysis_frame(self):
        for widget in self.analysis_frame.winfo_children():
            widget.destroy()
    
    def run(self):
        # Configure ttk styles
        style = ttk.Style()
        style.theme_use('clam')
        
        # Configure custom styles
        style.configure('Secondary.TButton',
                       background=self.colors['light_gray'],
                       foreground=self.colors['dark_gray'],
                       font=('Segoe UI', 9),
                       borderwidth=1,
                       relief='flat')
        
        style.map('Secondary.TButton',
                 background=[('active', self.colors['concrete_gray']),
                           ('pressed', self.colors['dark_gray'])])
        
        style.configure('Action.TButton',
                       background=self.colors['primary'],
                       foreground='white',
                       font=('Segoe UI', 9),
                       borderwidth=1,
                       relief='flat')
        
        style.map('Action.TButton',
                 background=[('active', self.colors['steel_blue']),
                           ('pressed', self.colors['secondary'])])
        
        print("\n" + "="*60)
        print("🚀 Starting Concrete Compressive Strength Predictor")
        print("="*60)
        print(f"📊 Target: {self.target_name}")
        print(f"🔢 Features: {len(self.feature_names)}")
        print(f"📈 Model R²: {self.r2_score:.4f}")
        print(f"🤖 Algorithm: Random Forest Regressor")
        print(f"⚙️ Hyperparameters: bootstrap=True, max_depth=10, min_samples_leaf=1,")
        print(f"                    min_samples_split=2, n_estimators=50")
        print("="*60)
        print("✅ Application ready!")
        
        # Auto-load a preset
        self.root.after(100, lambda: self.load_preset_by_name("Standard Concrete"))
        
        self.root.mainloop()

# Run the application
if __name__ == "__main__":
    try:
        print("Initializing Concrete Strength Predictor...")
        app = ConcreteStrengthPredictor()
        app.run()
    except Exception as e:
        print(f"❌ Application Error: {str(e)}")
        messagebox.showerror("Application Error", f"Failed to start:\n\n{str(e)}")

Initializing Concrete Strength Predictor...
📂 Loading data from: D:\2026 Work\My papers\Paper 1\Data.csv
✅ Dataset loaded successfully!
📐 Dataset shape: (299, 17)
🔧 Cleaned columns: ['OPC_kg_m3', 'S_kg_m3', 'W_B', 'FA_kg_m3', 'GS_kg_m3', 'SF_kg_m3', 'SP_kg_m3', 'HPMC_kg_m3', 'W_kg_m3', 'Vf', 'CA_days', 'LD_x_y_z', 'Lf_Df', 'Df_mm', 'Lf_mm', 'Ftype', 'CS_MPa']
🎯 Target column: 'CS_MPa'

📋 Features found (16):
    1. OPC_kg_m3
    2. S_kg_m3
    3. W_B
    4. FA_kg_m3
    5. GS_kg_m3
    6. SF_kg_m3
    7. SP_kg_m3
    8. HPMC_kg_m3
    9. W_kg_m3
   10. Vf
   11. CA_days
   12. LD_x_y_z
   13. Lf_Df
   14. Df_mm
   15. Lf_mm
   16. Ftype
🔍 Feature statistics calculated for 16 features

🤖 Training Random Forest model with hyperparameters:
  • Bootstrap: True
  • Max Depth: 10
  • Min Samples Leaf: 1
  • Min Samples Split: 2
  • N Estimators: 50
  • Random State: 42
✅ Random Forest model training completed!

🏆 Top 5 Most Important Features:
   1. SF_kg_m3                       : 0.5802
  